<a href="https://colab.research.google.com/github/ZilongJi/NEUR0016Lectures/blob/main/HPCSpatialRepresentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1, Let's download the dataset from the GitHub repo

In [2]:
# Clone the entire GitHub repository
!git clone https://github.com/ZilongJi/NEUR0016Lectures.git

# Change working directory to the cloned folder
%cd NEUR0016Lectures

Cloning into 'NEUR0016Lectures'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 1.56 MiB | 10.26 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/NEUR0016Lectures


### 2, Let's plot some receptive fields of place cells!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = np.load(save_path)
SpikeArray = data["SpikeArray"]
position = data["position"]
timestamps = data["timestamps"]
samplerate = data["samplerate"]